# Analysis of hypertension from UKBB

- This notebook applies the Get_Job_Script.ipynb to automatically generate the sbatch scripts to run in Yale's cluster. The end result is to apply various LMM workflows to perform association analysis in different hearing impairment traits, do clumping analysis and extract associated regions.

## File paths on Yale cluster

- Genotype files in PLINK format: `/gpfs/gibbs/pi/dewan/data/UKBiobank/genotype_files/pleiotropy_geneticfiles/UKB_Caucasians_phenotypeindepqc120319_updated020720removedwithdrawnindiv`

- Genotype files in bgen format: 
`SAY/dbgapstg/scratch/UKBiobank/genotype_files/ukb39554_imputeddataset/`

- Summary stats for imputed variants BOLT-LMM: `/gpfs/gibbs/pi/dewan/data/UKBiobank/results/BOLTLMM_results/results_imputed_data`
- Summary stats for inputed variants FastGWA: `/gpfs/gibbs/pi/dewan/data/UKBiobank/results/FastGWA_results/results_imputed_data`

- Phenotype files: `/gpfs/gibbs/pi/dewan/data/UKBiobank/phenotype_files/pleiotropy_R01/phenotypesforanalysis/UKB_Caucasiansubset_HYPERTENSION_casesbyICDorfield20002or6150ormedicationf6153or6177or20003orbybpreadings_081220`

 `/gpfs/gibbs/pi/dewan/data/UKBiobank/phenotype_files/pleiotropy_R01/phenotypesforanalysis/UKB_Caucasiansubset_HYPERTENSION_casesbyICDorfield20002or6150ormedicationf6153or6177or20003_notbybpreadings_081220`

- Relationship file: `/gpfs/gibbs/pi/dewan/data/UKBiobank/genotype_files/pleiotropy_geneticfiles/unrelated_n307259/UKB_unrelatedcauc_phenotypes_asthmat2dbmiwaisthip_agesex_waisthipratio_040620`

## Bash variables for workflow configuration for the hypertension (first phenotype)

In [4]:
# Common variables
tpl_file=../farnam.yml
bfile=/gpfs/gibbs/pi/dewan/data/UKBiobank/genotype_files/pleiotropy_geneticfiles/UKB_Caucasians_phenotypeindepqc120319_updated082020removedwithdrawnindiv.bed
sampleFile=/gpfs/gibbs/pi/dewan/data/UKBiobank/genotype_files/ukb39554_imputeddataset/ukb32285_imputedindiv.sample
bgenFile=`echo /gpfs/gibbs/pi/dewan/data/UKBiobank/genotype_files/ukb39554_imputeddataset/ukb_imp_chr{1..22}_v3.bgen`
unrelated_samples=/gpfs/gibbs/pi/dewan/data/UKBiobank/genotype_files/pleiotropy_geneticfiles/unrelated_n307259/UKB_unrelatedcauc_phenotypes_asthmat2dbmiwaisthip_agesex_waisthipratio_040620
formatFile_fastgwa=~/project/UKBB_GWAS_dev/data/fastGWA_template.yml
formatFile_bolt=~/project/UKBB_GWAS_dev/data/boltlmm_template.yml
formatFile_saige=~/project/UKBB_GWAS_dev/data/saige_template.yml
# Container
container_lmm=/gpfs/gibbs/pi/dewan/data/UKBiobank/lmm.sif
container_marp=/gpfs/gibbs/pi/dewan/data/UKBiobank/marp.sif
# LMM directories
lmm_dir_fastgwa=/gpfs/gibbs/pi/dewan/data/UKBiobank/results/FastGWA_results/results_imputed_data/
lmm_dir_bolt=/gpfs/gibbs/pi/dewan/data/UKBiobank/results/BOLTLMM_results/results_imputed_data/
lmm_dir_saige=/gpfs/gibbs/pi/dewan/data/UKBiobank/results/SAIGE_results/results_imputed_data/
lmm_sos=~/project/bioworkflows/GWAS/LMM.ipynb
lmm_sbatch_fastgwa=../output/$(date +"%Y-%m-%d")_imp-fastgwa.sbatch
lmm_sbatch_bolt=../output/$(date +"%Y-%m-%d")_imp-bolt.sbatch
lmm_sbatch_saige=../output/$(date +"%Y-%m-%d")_imp-saige.sbatch
#Phenotype file
phenoFile=/gpfs/gibbs/pi/dewan/data/UKBiobank/phenotype_files/pleiotropy_R01/phenotypesforanalysis/UKB_Caucasiansubset_HYPERTENSION_casesbyICDorfield20002or6150ormedicationf6153or6177or20003orbybpreadings_081220
## LMM variables 
LDscoresFile=/gpfs/gibbs/pi/dewan/data/UKBiobank/LDSCORE.1000G_EUR.tab.gz
geneticMapFile=/gpfs/gibbs/pi/dewan/data/UKBiobank/genetic_map_hg19_withX.txt.gz
covarMaxLevels=10
numThreads=20
bgenMinMAF=0.001
bgenMinINFO=0.8
lmm_job_size=1
ylim=0
### Specific to FastGWA
grmFile=/gpfs/gibbs/pi/dewan/data/UKBiobank/results/FastGWA_results/results_imputed_data/UKB_Caucasians_phenotypeindepqc120319_updated020720removedwithdrawnindiv.grm.sp
### Specific to SAIGE
bgenMinMAC=4
trait_type=quantitative
loco=TRUE
sampleCol=IID
# LD clumping directories
clumping_dir=/gpfs/gibbs/pi/dewan/data/UKBiobank/results/LD_clumping/
clumping_sos=~/project/bioworkflows/GWAS/LD_Clumping.ipynb
clumping_sbatch=../output/$(date +"%Y-%m-%d")_hypertension_imp_ldclumping.sbatch
## LD clumping variables
# For sumtastsFiles if more than one provide each path
bfile_ref=/gpfs/gibbs/pi/dewan/data/UKBiobank/results/LD_clumping/UKB_Caucasians_phenotypeindepqc120319_updated020720removedwithdrawnindiv.1210.ref_geno.bed
# Changes dependending upon which traits are analyzed
sumstatsFiles=/gpfs/gibbs/pi/dewan/data/UKBiobank/results/FastGWA_results/results_imputed_data/hypertension_BP/*.snp_stats.gz
ld_sample_size=1210
clump_field=P
clump_p1=5e-08
clump_p2=1
clump_r2=0.2
clump_kb=2000
clump_annotate=BP
numThreads=20
clump_job_size=1
#clumpFile= 
#clumregionFile=
# Region extraction directories
extract_dir=/gpfs/gibbs/pi/dewan/data/UKBiobank/results/region_extraction/hypertension_BP
extract_sos=~/project/bioworkflows/GWAS/Region_Extraction.ipynb
extract_sbatch=../output/$(date +"%Y-%m-%d")_hypertension_BP_imp-region.sbatch
## Region extraction variables
region_file=/gpfs/gibbs/pi/dewan/data/UKBiobank/results/LD_clumping/hypertension_BP/*.clumped_region
geno_path=/gpfs/gibbs/pi/dewan/data/UKBiobank/results/UKBB_bgenfilepath.txt
sumstats_path=/gpfs/gibbs/pi/dewan/data/UKBiobank/results/FastGWA_results/results_imputed_data/hypertension_BP/*.snp_stats.gz
extract_job_size=10

# FastGWA jobs

# Hypertension (with BP cutoff)

In [2]:
lmm_dir_fastgwa=/gpfs/gibbs/pi/dewan/data/UKBiobank/results/FastGWA_results/results_imputed_data/hypertension_BP
lmm_sbatch_fastgwa=../output/$(date +"%Y-%m-%d")_hypertension_BP_imp-fastgwa.sbatch
phenoFile=/gpfs/gibbs/pi/dewan/data/UKBiobank/phenotype_files/pleiotropy_R01/phenotypesforanalysis/UKB_Caucasiansubset_HYPERTENSION_casesbyICDorfield20002or6150ormedicationf6153or6177or20003orbybpreadings_081220
covarFile=/gpfs/gibbs/pi/dewan/data/UKBiobank/phenotype_files/pleiotropy_R01/phenotypesforanalysis/UKB_Caucasiansubset_HYPERTENSION_casesbyICDorfield20002or6150ormedicationf6153or6177or20003orbybpreadings_081220
phenoCol="case_byICDorfield20002or6150ormedicationf6153or6177or20003orbybpreadings"
covarCol="SEX smoking1 smoking2 Alcohol1 Alcohol2 Alcohol3 Alcohol4"
qCovarCol="AGE"

lmm_args="""fastGWA
    --cwd $lmm_dir_fastgwa 
    --bfile $bfile 
    --sampleFile $sampleFile
    --bgenFile $bgenFile 
    --phenoFile $phenoFile 
    --formatFile $formatFile_fastgwa 
    --covarFile $covarFile  
    --phenoCol $phenoCol 
    --covarCol $covarCol 
    --covarMaxLevels $covarMaxLevels 
    --qCovarCol $qCovarCol 
    --numThreads $numThreads 
    --bgenMinMAF $bgenMinMAF 
    --bgenMinINFO $bgenMinINFO 
    --job_size $lmm_job_size
    --grmFile $grmFile
    --ylim $ylim
    --container_lmm $container_lmm
    --container_marp $container_marp
"""

sos run ~/project/bioworkflows/GWAS/Get_Job_Script.ipynb farnam \
    --template-file $tpl_file \
    --workflow-file $lmm_sos \
    --to-script $lmm_sbatch_fastgwa \
    --args "$lmm_args"

INFO: Running farnam: Configuration for Yale `farnam` cluster
INFO: farnam is completed.
INFO: farnam output:   ../output/2020-10-02_hypertension_BP_imp-fastgwa.sbatch
INFO: Workflow farnam (ID=9a7570a4a35efc79) is executed successfully with 1 completed step.



# Hypertension (without BP cutoff)

In [3]:
lmm_dir_fastgwa=/gpfs/gibbs/pi/dewan/data/UKBiobank/results/FastGWA_results/results_imputed_data/hypertension_notBP
lmm_sbatch_fastgwa=../output/$(date +"%Y-%m-%d")_hypertension_notBP_imp-fastgwa.sbatch
phenoFile=/gpfs/gibbs/pi/dewan/data/UKBiobank/phenotype_files/pleiotropy_R01/phenotypesforanalysis/UKB_Caucasiansubset_HYPERTENSION_casesbyICDorfield20002or6150ormedicationf6153or6177or20003_notbybpreadings_081220
covarFile=/gpfs/gibbs/pi/dewan/data/UKBiobank/phenotype_files/pleiotropy_R01/phenotypesforanalysis/UKB_Caucasiansubset_HYPERTENSION_casesbyICDorfield20002or6150ormedicationf6153or6177or20003_notbybpreadings_081220
phenoCol="case_byICDorfield20002or6150ormedicationf6153or6177or20003_notbybp_controlsremovemiddlebp"
covarCol="SEX smoking1 smoking2 Alcohol1 Alcohol2 Alcohol3 Alcohol4"
qCovarCol="AGE"

lmm_args="""fastGWA
    --cwd $lmm_dir_fastgwa 
    --bfile $bfile 
    --sampleFile $sampleFile
    --bgenFile $bgenFile 
    --phenoFile $phenoFile 
    --formatFile $formatFile_fastgwa 
    --covarFile $covarFile  
    --phenoCol $phenoCol 
    --covarCol $covarCol 
    --covarMaxLevels $covarMaxLevels 
    --qCovarCol $qCovarCol 
    --numThreads $numThreads 
    --bgenMinMAF $bgenMinMAF 
    --bgenMinINFO $bgenMinINFO 
    --job_size $lmm_job_size
    --grmFile $grmFile
    --ylim $ylim
    --container_lmm $container_lmm
    --container_marp $container_marp
"""

sos run ~/project/bioworkflows/GWAS/Get_Job_Script.ipynb farnam \
    --template-file $tpl_file \
    --workflow-file $lmm_sos \
    --to-script $lmm_sbatch_fastgwa \
    --args "$lmm_args"

INFO: Running farnam: Configuration for Yale `farnam` cluster
INFO: farnam is completed.
INFO: farnam output:   ../output/2020-10-02_hypertension_notBP_imp-fastgwa.sbatch
INFO: Workflow farnam (ID=4231e6ebb2adcd06) is executed successfully with 1 completed step.



# Bolt lmm (with BP)

In [5]:
lmm_dir_bolt=/gpfs/gibbs/pi/dewan/data/UKBiobank/results/BOLTLMM_results/results_imputed_data/hypertension_BP
lmm_sbatch_bolt=../output/$(date +"%Y-%m-%d")_hypertension_BP_imp-bolt.sbatch
phenoCol="case_byICDorfield20002or6150ormedicationf6153or6177or20003orbybpreadings"
covarCol="SEX smoking1 smoking2 Alcohol1 Alcohol2 Alcohol3 Alcohol4"
qcovarCol="AGE"

lmm_args="""boltlmm
    --cwd $lmm_dir_bolt 
    --bfile $bfile 
    --sampleFile $sampleFile
    --bgenFile $bgenFile 
    --phenoFile $phenoFile 
    --formatFile $formatFile_bolt 
    --covarFile $covarFile 
    --LDscoresFile $LDscoresFile 
    --geneticMapFile $geneticMapFile 
    --phenoCol $phenoCol 
    --covarCol $covarCol 
    --covarMaxLevels $covarMaxLevels 
    --qCovarCol $qCovarCol 
    --numThreads $numThreads 
    --bgenMinMAF $bgenMinMAF 
    --bgenMinINFO $bgenMinINFO 
    --job_size $lmm_job_size
    --ylim $ylim
    --container_lmm $container_lmm
    --container_marp $container_marp
"""

sos run ~/project/bioworkflows/GWAS/Get_Job_Script.ipynb farnam \
    --template-file $tpl_file \
    --workflow-file $lmm_sos \
    --to-script $lmm_sbatch_bolt \
    --args "$lmm_args"

INFO: Running farnam: Configuration for Yale `farnam` cluster
INFO: farnam is completed.
INFO: farnam output:   ../output/2020-10-30_hypertension_BP_imp-bolt.sbatch
INFO: Workflow farnam (ID=3d1bb42cce2dbe73) is executed successfully with 1 completed step.



# BP (without BP)

In [7]:
lmm_dir_bolt=/gpfs/gibbs/pi/dewan/data/UKBiobank/results/BOLTLMM_results/results_imputed_data/hypertension_notBP
lmm_sbatch_bolt=../output/$(date +"%Y-%m-%d")_hypertension_notBP_imp-bolt.sbatch
phenoCol="case_byICDorfield20002or6150ormedicationf6153or6177or20003_notbybp_controlsremovemiddlebp"
covarCol="SEX smoking1 smoking2 Alcohol1 Alcohol2 Alcohol3 Alcohol4"
qCovarCol="AGE"

lmm_args="""boltlmm
    --cwd $lmm_dir_bolt 
    --bfile $bfile 
    --sampleFile $sampleFile
    --bgenFile $bgenFile 
    --phenoFile $phenoFile 
    --formatFile $formatFile_bolt 
    --covarFile $covarFile 
    --LDscoresFile $LDscoresFile 
    --geneticMapFile $geneticMapFile 
    --phenoCol $phenoCol 
    --covarCol $covarCol 
    --covarMaxLevels $covarMaxLevels 
    --qCovarCol $qCovarCol 
    --numThreads $numThreads 
    --bgenMinMAF $bgenMinMAF 
    --bgenMinINFO $bgenMinINFO 
    --job_size $lmm_job_size
    --ylim $ylim
    --container_lmm $container_lmm
    --container_marp $container_marp
"""

sos run ~/project/bioworkflows/GWAS/Get_Job_Script.ipynb farnam \
    --template-file $tpl_file \
    --workflow-file $lmm_sos \
    --to-script $lmm_sbatch_bolt \
    --args "$lmm_args"

INFO: Running farnam: Configuration for Yale `farnam` cluster
INFO: farnam is completed.
INFO: farnam output:   ../output/2020-10-30_hypertension_notBP_imp-bolt.sbatch
INFO: Workflow farnam (ID=61c21502bda109e6) is executed successfully with 1 completed step.

